In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[4],3b
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[6]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.000126                    1  0.000101
0.000750        2  0.000134                    2  0.000136
0.001052        3  0.000150                    3  0.000101
0.001476        4  0.000167                    4  0.000112
0.002070        5  0.000185                    5  0.000124
0.002904        6  0.000204                    6  0.000137
0.004074        7  0.000223                    7  0.000150
0.005714        8  0.000242                    8  0.000164
0.008015        9  0.000263                    9  0.000178
0.011243       10  0.000284                   10  0.000193
0.015771       11  0.000306                   11  0.000208
0.022122       12  0.000328                   12  0.000224
0.031031       13  0.000352                   13  0.000241
0.043528       14  0.000383                   14  0.000268
0.061057       15  0.000502                   15  0.000352
0.085645       16  0.000795                   16  0.000554
0.120136       17  0.001255                   17  0.000872
0.168516       18  0.001877                   18  0.001303
0.236378       19  0.002702                   19  0.001873
0.331549       20  0.003778                   20  0.002616
0.465100       21  0.005166                   21  0.003596
0.652400       22  0.006738                   22  0.004769
0.915100       23  0.007284                   23  0.005234
1.283650       24  0.006592                   24  0.004815
1.800600       25  0.005536                   25  0.004118
2.525700       26  0.004332                   26  0.003294
3.542800       27  0.003297                   27  0.002563
4.969550       28  0.002480                   28  0.001969
6.970850       29  0.001842                   29  0.001488
9.778100       30  0.001349                   30  0.001100
13.715850      31  0.000969                   31  0.000789
19.239350      32  0.000675                   32  0.000542
26.987250      33  0.000446                   33  0.000346
37.855300      34  0.000260                   34  0.000188
53.100050      35  0.000069                   35  0.000036
73.887500      36 -0.000084                   36 -0.000087
97.662500      37 -0.000144                   37 -0.000138
121.437500     38 -0.000118                   38 -0.000119
145.212500     39 -0.000010                   39 -0.000035
168.987500     40  0.000163                   40  0.000098
192.762500     41  0.000571                   41  0.000418
216.537500     42  0.002126                   42  0.001723
240.312500     43  0.005712                   43  0.005083
264.087500     44  0.012184                   44  0.011740
287.862500     45  0.021898                   45  0.022140
311.637500     46  0.034684                   46  0.035363
335.412500     47  0.050020                   47  0.050270
359.187500     48  0.066521                   48  0.067282
382.962500     49  0.080351                   49  0.082524
406.737500     50  0.096366                   50  0.097688
430.512500     51  0.114525                   51  0.111375
454.287500     52  0.134803                   52  0.125876
478.062500     53  0.157161                   53  0.144425
501.837500     54  0.181221                   54  0.167874
525.612500     55  0.207107                   55  0.195138
549.387500     56  0.234232                   56  0.222508
573.162500     57  0.262301                   57  0.247879
596.937500     58  0.291114                   58  0.271233
620.712500     59  0.320073                   59  0.294462
644.487500     60  0.349238                   60  0.321093
668.262500     61  0.377849                   61  0.352394
692.037500     62  0.403856                   62  0.385726
715.812500     63  0.425231                   63  0.416765
739.587500     64  0.466503                   64  0.468097
763.362500     65  0.487738                   65  0.497433
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -39.984950  0.000000e+00 -39.984950 -40.085075   
0.000624    2     -39.984950  4.137529e-08 -39.984950 -40.085075   
0.000876    3     -39.984950  5.826523e-08 -39.984950 -40.085075   
0.001229    4     -39.984950  8.261300e-08 -39.984950 -40.085075   
0.001723    5     -39.984950  1.177405e-07 -39.984950 -40.085075   
0.002417    6     -39.984950  1.684073e-07 -39.984950 -40.085075   
0.003391    7     -39.984950  2.414862e-07 -39.984950 -40.085075   
0.004757    8     -39.984950  3.468836e-07 -39.984950 -40.085075   
0.006672    9     -39.984951  4.988721e-07 -39.984950 -40.085076   
0.009359    10    -39.984951  7.180364e-07 -39.984950 -40.085076   
0.013128    11    -39.984951  1.034040e-06 -39.984950 -40.085076   
0.018415    12    -39.984951  1.489645e-06 -39.984950 -40.085076   
0.025830    13    -39.984952  2.146479e-06 -39.984949 -40.085076   
0.036232    14    -39.984952  3.093448e-06 -39.984949 -40.085077   
0.050823    15    -39.984953  4.468899e-06 -39.984948 -40.085077   
0.071291    16    -39.984954  6.646114e-06 -39.984947 -40.085078   
0.100000    17    -39.984955  1.050947e-05 -39.984944 -40.085078   
0.140271    18    -39.984956  1.760401e-05 -39.984938 -40.085079   
0.196760    19    -39.984956  3.056297e-05 -39.984926 -40.085079   
0.275997    20    -39.984955  5.405465e-05 -39.984900 -40.085078   
0.387100    21    -39.984947  9.630041e-05 -39.984851 -40.085072   
0.543100    22    -39.984927  1.717277e-04 -39.984755 -40.085058   
0.761700    23    -39.984882  3.017303e-04 -39.984581 -40.085028   
1.068500    24    -39.984810  4.937324e-04 -39.984316 -40.084978   
1.498800    25    -39.984718  7.371507e-04 -39.983981 -40.084917   
2.102400    26    -39.984611  1.026345e-03 -39.983584 -40.084850   
2.949000    27    -39.984501  1.350475e-03 -39.983150 -40.084786   
4.136600    28    -39.984393  1.707029e-03 -39.982686 -40.084733   
5.802500    29    -39.984296  2.098453e-03 -39.982197 -40.084699   
8.139200    30    -39.984216  2.529164e-03 -39.981687 -40.084694   
11.417000   31    -39.984169  3.006261e-03 -39.981163 -40.084733   
16.014700   32    -39.984175  3.539964e-03 -39.980635 -40.084835   
22.464000   33    -39.984264  4.143536e-03 -39.980120 -40.085027   
31.510500   34    -39.984475  4.832173e-03 -39.979642 -40.085342   
44.200100   35    -39.984868  5.615968e-03 -39.979252 -40.085824   
62.000000   36    -39.985535  6.430029e-03 -39.979106 -40.086547   
85.775000   37    -39.986506  7.164011e-03 -39.979343 -40.087532   
109.550000  38    -39.987505  7.755381e-03 -39.979750 -40.088514   
133.325000  39    -39.988579  8.498757e-03 -39.980081 -40.089558   
157.100000  40    -39.989780  9.669928e-03 -39.980110 -40.090716   
180.875000  41    -39.991194  1.154248e-02 -39.979651 -40.092077   
204.650000  42    -39.993333  1.528853e-02 -39.978044 -40.094162   
228.425000  43    -39.998184  2.612860e-02 -39.972056 -40.099011   
252.200000  44    -40.007636  5.166858e-02 -39.955968 -40.108615   
275.975000  45    -40.024049  1.023950e-01 -39.921655 -40.125419   
299.750000  46    -40.050132  1.901477e-01 -39.859984 -40.152182   
323.525000  47    -40.088814  3.265116e-01 -39.762303 -40.191833   
347.300000  48    -40.143213  5.217804e-01 -39.621432 -40.247294   
371.075000  49    -40.215760  7.816723e-01 -39.434087 -40.320870   
394.850000  50    -40.304050  1.096255e+00 -39.207794 -40.409962   
418.625000  51    -40.409251  1.472855e+00 -38.936396 -40.515533   
442.400000  52    -40.532308  1.918450e+00 -38.613859 -40.638322   
466.175000  53    -40.673920  2.439708e+00 -38.234212 -40.779086   
489.950000  54    -40.834570  3.042973e+00 -37.791597 -40.938174   
513.725000  55    -41.014201  3.732979e+00 -37.281221 -41.115681   
537.500000  56    -41.212711  4.514768e+00 -36.697943 -41.311327   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')